In [1]:
import svfpy
# import rasterio
import numpy as np
from rioxarray import merge
from scipy import ndimage
import xarray as xr
from rasterio.transform import Affine
# import os

In [2]:
# mds = '../test/data/mds_sao_paulo_city_1000.tiff'
# mds = '../data/raster/MDS_sampa-ZSTD.tif'
mds = '../tmp/raster_mdt_bhm.tiff'

In [3]:
svf = svfpy.SVF(mds, '../tmp/', kernel_size_side=2500)

In [4]:
svf.downscales()

array([ 1.,  1.,  2.,  2.,  4.,  6., 10., 31.])

In [5]:
svf.tangents[::-1] * (1/svf.resolution)

array([ 0.74236858,  1.43498139,  2.11253637,  2.93972368,  4.11071273,
        6.07947366, 10.4774891 , 31.93743885])

In [6]:
svf.pad_kernel

518.0

In [7]:
seila = svf.prepare_downscales()

Preparing Downscale, resolution of 0.5
Preparing Downscale, resolution of 1.0
Preparing Downscale, resolution of 2.0
Preparing Downscale, resolution of 3.0
Preparing Downscale, resolution of 5.0
Preparing Downscale, resolution of 15.5


In [8]:
svf.kernels(resolution=0.5)

(2, 3)

In [9]:
k,m = svf.working_kernel(0,1,0.5)

In [10]:
[(svf.kernels(r)[0] + 1) * (svf.kernels(r)[1] + 1)  for r in svf.resolutions]

[12, 4, 1, 1, 1, 1]

In [11]:
[svf.kernels(r)  for r in svf.resolutions]

[(2, 3), (1, 1), (0, 0), (0, 0), (0, 0), (0, 0)]

In [12]:
svf.resolutions

array([ 0.5,  1. ,  2. ,  3. ,  5. , 15.5])

In [13]:
svf.svf()

0 0 0.5
0 1 0.5


In [14]:
svf.xmds.shape

(1, 7483, 8624)

In [14]:
# [svf._agg_scale(res) for res in svf.resolutions]

In [15]:
# svf._agg_all()

## Validação

In [14]:
import geopandas as gpd

In [15]:
quantidade_pontos = 10

In [16]:
col_idx = np.random.randint(svf.xmds.rio.width, size=quantidade_pontos)
row_idx = np.random.randint(svf.xmds.rio.height, size=quantidade_pontos)

In [17]:
points = np.concatenate((np.expand_dims(col_idx, axis=1), np.expand_dims(row_idx, axis=1)), axis=1)

In [18]:
svf.xmds.rio.width, svf.xmds.rio.height

(8624, 7483)

In [19]:
# points = [[25,100], [25,101], [25,102],[25,103],[25,104],[25,105],[25,106],[25,107],[25,108],[25,109],]

In [20]:
points_xy = [list(p) * svf.xmds.rio.transform() for p in points]

/tmp/ipykernel_34935/2938777973.py:1: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  points_xy = [list(p) * svf.xmds.rio.transform() for p in points]


In [21]:
points_xy = np.array(points_xy) + [svf.resolution/2, svf.resolution/2]

In [22]:
len(points), len(points_xy)

(10, 10)

In [23]:
points_xy

array([[ 330810.25      , 7390647.62992106],
       [ 331424.75      , 7390674.62992106],
       [ 334181.25      , 7392006.62992106],
       [ 332213.25      , 7390525.62992106],
       [ 331954.25      , 7391530.62992106],
       [ 334352.25      , 7392120.12992106],
       [ 333510.75      , 7392383.12992106],
       [ 332441.25      , 7390645.62992106],
       [ 332859.25      , 7392690.62992106],
       [ 330480.75      , 7391339.62992106]])

In [24]:
points_svf = np.zeros(quantidade_pontos, dtype='float')

for i in range(quantidade_pontos):
    col, row = points[i]
    points_svf[i] = svf.calc_svf_point(row,col)
    print(f'{i}', end=". ")

0. 1. 2. 3. 4. 5. 6. 7. 8. 9. 

In [25]:
# svf.calc_svf_point(8212,3076)

In [26]:
gdf_svf = gpd.GeoDataFrame({'svf':points_svf}, geometry=gpd.points_from_xy(points_xy[:, 0], points_xy[:, 1], crs='EPSG:31983'))

In [27]:
gdf_svf.to_file('../validation/sp_validation.gpkg', driver='GPKG')

In [28]:
svf.xmds.shape

(1, 7483, 8624)